In [2]:
import tensorflow as tf

# Download pre-trained InceptionV3 model
inceptionv3 = tf.keras.applications.InceptionV3(weights='imagenet')



In [3]:
# Load the pre-trained inceptionv3 model

import tensorflow as tf

# Load pre-trained InceptionV3 model without top classification layer
inceptionv3 = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)

# Remove the last layer from the model
inceptionv3 = tf.keras.Model(inputs=inceptionv3.input, outputs=inceptionv3.layers[-1].output)



In [4]:
inceptionv3

In [5]:
# Define the input size of the VGG16 model
input_size = (224, 224)

# Specify the path to the PASCAL VOC 2007 dataset
data = 'C:\\Users\\Admin\\Downloads\\VOCdevkit\\VOC2007'

In [6]:
import os
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [7]:
# Specify the path to the training image folder
train_image_folder = os.path.join(data, 'JPEGImages')
train_image_folder

'C:\\Users\\Admin\\Downloads\\VOCdevkit\\VOC2007\\JPEGImages'

In [8]:
# Get the list of image filenames in the training set
train_image_filenames = os.listdir(train_image_folder)

In [9]:
# Load and preprocess the images in the PASCAL VOC 2007 training set
def preprocess_image(image_path):
    img = load_img(image_path, target_size=input_size)
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# Process each image in the training set and extract features
training_features = []
for filename in train_image_filenames:
    image_path = os.path.join(train_image_folder, filename)
    img = preprocess_image(image_path)
    features = inceptionv3.predict(tf.expand_dims(img, axis=0))
    training_features.append(features)

# Concatenate the extracted features into a single array
training_features = tf.concat(training_features, axis=0)

1/1 [==============================] - 3s 3s/step


In [10]:
# Save the features to a NumPy array
np.save('features.npy', training_features)

In [11]:
# Specify the path to the annotations folder
annotation_folder = os.path.join(data, 'Annotations')

In [12]:
# Get the list of XML annotation filenames
annotation_filenames = os.listdir(annotation_folder)

In [13]:
# Process each XML annotation file and extract the label names

import xml.etree.ElementTree as ET

label_names = []
for filename in annotation_filenames:
    annotation_path = os.path.join(annotation_folder, filename)
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        label_names.append(class_name)

# Get unique label names
unique_label_names = list(set(label_names))

In [24]:
# Create a dictionary to map unique label names to label indices
class_to_label = {label_name: label_index for label_index, label_name in enumerate(unique_label_names)}

# Process each XML annotation file again and extract the labels
labels = []
labels_bin = []
for filename in annotation_filenames:
    labels=[]
    annotation_path = os.path.join(annotation_folder, filename)
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        label = class_to_label[class_name]
        labels.append(label)
    #test.append(labels)
    labels_bin.append(labels)
     
    
print(labels_bin)

[[7, 15], [12], [11, 6], [3, 3, 3, 3, 3, 3, 3], [0, 9, 6, 6, 6, 6, 6, 6], [6], [16, 15], [18], [4], [17, 3, 3, 15, 15, 15, 3], [8], [7], [16, 15], [4, 4, 4, 15, 15, 15, 4, 15, 15, 15, 15], [15], [18], [7], [12], [7], [8, 14, 15], [10], [15, 15, 15], [5], [18], [18], [17], [14, 15, 15, 15], [16], [10], [15, 15, 15, 14], [5, 5, 15, 15, 15, 6], [1], [13], [10], [19, 15], [0, 0], [3], [3], [7], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [7], [19, 19, 19, 19, 19, 19, 19, 19, 19], [3, 3], [6, 6, 6, 6, 6, 11, 9], [6, 9, 15, 15], [7, 7, 7], [12], [12], [6, 15, 15], [18], [14, 14], [15, 15], [11, 6, 15], [18], [12], [3, 3, 3, 3, 3, 3, 3, 3, 3], [19, 15, 15], [10], [7, 7, 11], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 8, 8, 8, 8, 8, 8, 8, 8, 6], [18], [10], [11, 6, 0, 0], [18], [15, 15, 15, 0, 6], [9, 9, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 15, 15, 15, 15], [10, 10, 10, 10, 15], [13, 15, 15, 15, 15, 15, 15, 15], [3], [18, 2], [3, 15, 15], [8, 15, 15], [6, 9, 2, 2, 15, 15, 15,

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
# Load the features and labels
features = np.load('features.npy')
labels = np.load('labels.npy')
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)


In [15]:
len(X_train), len(X_val), len(y_train), len(y_val)

(3961, 991, 3961, 991)

In [16]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((3961, 5, 5, 2048), (991, 5, 5, 2048), (3961, 20), (991, 20))

In [23]:
# Reshape training data
n_train_samples = X_train.shape[0]
X_train_2d = X_train.reshape(n_train_samples, -1)  # Reshape to (n_train_samples, 5*5*2048)
y_train_2d = y_train.argmax(axis=1)  # Convert one-hot encoded labels to single dimension

# Reshape validation data
n_val_samples = X_val.shape[0]
X_val_2d = X_val.reshape(n_val_samples, -1)  # Reshape to (n_val_samples, 5*5*2048)
y_val_2d = y_val.argmax(axis=1)  # Convert one-hot encoded labels to single dimension

In [25]:
X_train_2d.shape, X_val_2d.shape, y_train_2d.shape, y_val_2d .shape

((3961, 51200), (991, 51200), (3961,), (991,))

In [33]:
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix



In [34]:


# Get the number of classes from the training labels
num_classes = len(np.unique(y_train_2d))

# Create a list to store the SVM classifiers
svm_classifiers = []

# Train binary SVM classifiers for each class
for class_label in range(num_classes):
    # Create an SVM classifier object
    svm_classifier = svm.SVC(kernel='linear')

    # Prepare binary labels for the current class
    binary_train_labels = (y_train_2d == class_label)
    binary_val_labels = (y_val_2d == class_label)

    # Train the SVM classifier
    svm_classifier.fit(X_train_2d, binary_train_labels)

    # Predict on the validation set
    val_predictions = svm_classifier.predict(X_val_2d)

    # Calculate accuracy
    accuracy = accuracy_score(binary_val_labels, val_predictions)
    print(f"Accuracy for class {class_label}: {accuracy}")

    # Store the trained classifier
    svm_classifiers.append(svm_classifier)


Accuracy for class 0: 0.9525731584258325
Accuracy for class 1: 0.9919273461150353
Accuracy for class 2: 0.9576185671039354
Accuracy for class 3: 0.941473259334006
Accuracy for class 4: 0.9818365287588294
Accuracy for class 5: 0.9757820383451059
Accuracy for class 6: 0.9162462159434914
Accuracy for class 7: 0.9626639757820383
Accuracy for class 8: 0.9828456104944501
Accuracy for class 9: 0.9889001009081736
Accuracy for class 10: 0.992936427850656
Accuracy for class 11: 0.9727547931382442
Accuracy for class 12: 0.987891019172553
Accuracy for class 13: 0.9889001009081736
Accuracy for class 14: 0.9899091826437941
Accuracy for class 15: 0.887991927346115
Accuracy for class 16: 0.987891019172553
Accuracy for class 17: 0.9899091826437941
Accuracy for class 18: 0.9828456104944501
Accuracy for class 19: 0.9868819374369324


In [35]:

# Calculate confusion matrix
val_predictions_all = []
for classifier in svm_classifiers:
    val_predictions_all.append(classifier.decision_function(X_val_2d))
val_predictions_all = np.column_stack(val_predictions_all)
confusion_matrix = np.argmax(val_predictions_all, axis=1)

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix)

 

Confusion Matrix:
[ 1 13 11  8 10  2 15 18  6 18 14 10  3 15 15  3 15 15  6 15  7  6  5  3
 15  3 15 10  2 19 13  6 19  1  3 15  3 10  7 18  7  7  3  2  3  7 16 12
  7  0  3 10 19  3  8  3 15  7 14  3  3 15  2  5  0  3 15 18 13 18  7  5
  3 11 10 19 16 15 15 12  2 13  2 18  3  7 11  4 15  4 17  5  7 12  6 15
  0 19 12 18  8  7  8 18  3  5  7  4  8 12  3  0  6  6  3  7  7 10 15  7
  8 18 10 15  8 15  5 15  3 14  3  6  3  6  3 16  7  6 10 15 18 12  3  2
 12  2 18 14 15 18 18 15 13 15 10 11  9  6  6 10 18  3 10 19 13 15 15  0
  3  8  8  7 18  7 15  6 15 15  2 11 14  5 18  3 15 13 12 12 15 12 15 10
  8  1 18 15  7 15  3  3  3  6  3  7 15  7 10  1  0  1  3 18  3  6 15  7
  6  6  1 18  3 12 12  8  7  0  3  3 15 18 10 10  3 15 12 15 11 15 18  6
  3  3  7 15 12  5  8 15  5 15  4 13 15  6 18 18  7  3  3 13  3  3 15 18
 10 15 11 12 19 15 15  3 14  5  2  8 11 15  1  3  6  5  6  7  3  4  3  3
 15  3  0 15 10  5  3  5  6  8 15 12 14  7 15  6 15 15  8  3  7 10 14 15
  3 15 15 18  3  3 15  4  1 15 15